The worflow:

for each frame:
    mask cars appearing on frame at instant t (maybe use dilation)
    extract edges (evaluate this two changing the order)
    get Lines filtered by certain range of angles (get horizontal and vertical, but separately) (maybe explore this for lane markings)
Once lines are saved:
        get histogram of orientations
        get histogram of lengths
        remove outliers
map all endpoints of lines to pre defined lengths
use them to train a MLP.
Test
    

In [3]:
import cv2
import numpy as np
import torchvision.transforms as T

from sklearn.neighbors import KernelDensity
from objectDetection import *
from carMasking import *
from imgFeatureUtils import *

%matplotlib inline

forExt = DLForegroundExtractor(512)
videoPath = "/home/hector/speed/speed_DS/v1.mp4"

In [4]:
yolo = objectDetector((512,512),
                       weightPath="yolov3/weights/yolov3.weights",
                       modelDef="yolov3/config/yolov3.cfg")
viz = objectVisualizer(classPath="yolov3/data/coco.names")
tools = objectFocusTools((512,512),yolo)
linesDataset = []
cap = cv2.VideoCapture("/home/hector/speed/speed_DS/v5.avi")
lineSet = []

if (cap.isOpened()== False): 
  print("Error opening video stream or file")
while(cap.isOpened()):
  ret, frame = cap.read()
  if ret == True:
    frame = cv2.resize(frame, (512,512))
    grayFrame = convertImg2Grayscale(frame)
#(-60,-40) vertical, 
    lines, edgeMap = tools.getObjectLinesAtFrame(frame,grayFrame,(0,20))
    lineDist = getLinesEuclideanDist(lines)
    thetas = getLinesThetas(lines)
    print(lines)
    lineSet.append(lines)
    empty = np.zeros((edgeMap.shape[0],edgeMap.shape[1],3))
    empty[:,:,0] = edgeMap
    #frameObjs = viz.drawBoxes(frame,detections)
    linesImg = drawLinesOnImg(empty, lines)
    cv2.imshow('Frame',linesImg)
    if cv2.waitKey(25) & 0xFF == ord('q'):
      break
  else: 
    break
featureSet = flatten(featureSet)
#print("final feature set")
#print(featureSet)
cap.release()
cv2.destroyAllWindows()
lineSet = flatten(lineSet)

[((400, 210), (381, 209))]
[((392, 229), (377, 227)), ((392, 229), (377, 228))]
[((400, 165), (385, 163)), ((400, 229), (380, 226)), ((396, 217), (381, 216)), ((206, 454), (176, 445)), ((201, 453), (185, 448))]
[((401, 164), (386, 162)), ((400, 227), (379, 224)), ((403, 210), (383, 207)), ((401, 206), (382, 205)), ((206, 442), (183, 435))]
[((223, 432), (199, 429)), ((402, 159), (386, 156)), ((402, 206), (385, 203)), ((399, 215), (383, 213)), ((401, 224), (382, 223)), ((119, 449), (101, 444)), ((115, 445), (100, 441)), ((98, 439), (80, 437))]
[((196, 460), (174, 452)), ((243, 423), (212, 421)), ((201, 465), (175, 456)), ((201, 464), (176, 456)), ((402, 158), (387, 155)), ((402, 225), (381, 221)), ((403, 204), (384, 203))]
[((243, 414), (221, 411)), ((203, 478), (188, 474)), ((122, 415), (103, 413)), ((137, 421), (118, 415))]
[((143, 408), (117, 402)), ((218, 440), (191, 431)), ((224, 445), (189, 433)), ((403, 221), (382, 218))]
[((225, 430), (200, 421)), ((228, 433), (213, 428)), ((142

NameError: name 'featureSet' is not defined

In [3]:
from projectiveGeometry import compute_edgelets, ransac_vanishing_point, vis_model
lineSet = np.expand_dims(lineSet, axis=0)[0]
edgelets = compute_edgelets(lineSet)
vp1 = ransac_vanishing_point(edgelets, num_ransac_iter=5000, 
                             threshold_inlier=5)
vis_model(empty, vp1)

ValueError: too many values to unpack (expected 2)

In [4]:
lineSet

array([list([((399, 167), (384, 164)), ((400, 210), (381, 209)), ((393, 221), (378, 219)), ((398, 233), (378, 229))]),
       list([((392, 229), (377, 228)), ((392, 229), (377, 227))]),
       list([((393, 228), (378, 226)), ((395, 224), (378, 220)), ((395, 217), (378, 216)), ((209, 454), (190, 450))]),
       list([((401, 159), (385, 158)), ((400, 227), (381, 224)), ((401, 206), (382, 205)), ((206, 442), (183, 435))]),
       list([((223, 432), (199, 429)), ((401, 158), (386, 156)), ((402, 224), (380, 223)), ((116, 449), (101, 444)), ((98, 439), (81, 437))]),
       list([((199, 464), (175, 456)), ((396, 215), (381, 214)), ((402, 225), (381, 221)), ((403, 203), (385, 202))]),
       list([((218, 413), (203, 410)), ((203, 478), (188, 474)), ((210, 451), (185, 442)), ((243, 414), (223, 411)), ((134, 419), (107, 413)), ((76, 487), (59, 484))]),
       list([((143, 408), (124, 403)), ((210, 440), (192, 434)), ((218, 440), (195, 432)), ((403, 221), (382, 219))]),
       list([((217, 478), 